In [6]:
import os

# Why Torch? You'll find the answer in the .md files! 
os.environ["KERAS_BACKEND"] = "torch"

import keras
import keras_tuner
from k_fold_cv.k_fold_generator import KFoldController
from utils.dataset_loader import dataset_loader
import keras_tuner
from k_fold_cv.k_fold_generator import KFoldController
from utils.dataset_loader import dataset_loader
import keras_tuner
from torch.utils.data import DataLoader
from torch.utils.data import DataLoader

In [7]:
from utils.my_tuner import HyperBandSaveLess
from models.tunable import Tunable
from models.simple_cnn.simple_cnn import SimpleCnnModelFamily

hyperparameters = keras_tuner.HyperParameters()
train, test = dataset_loader((224, 224), is_grayscale=False)

dataset_split_controller = KFoldController(5)
dataset_split_controller.load_data(train)

local_train, validation = dataset_split_controller.get_data_for_fold(0)
train_dataloader = DataLoader(dataset=local_train, batch_size=16, shuffle=True)
validation_dataloader = DataLoader(dataset=validation, batch_size=16, shuffle=True)

hypermodel = SimpleCnnModelFamily()
tunableModel = Tunable((3,224,224), hypermodel)

hyperparameters.Int("conv_layers", min_value=1, max_value=2) # We only want one Conv layer 
hyperparameters.Fixed("optimizer", "adam")

#todo change name it is not hyperband as we have to avoid disk usage problem
tuner = HyperBandSaveLess(
    tunableModel,
    hyperparameters=hyperparameters,
    objective='val_loss',
    tune_new_entries=True,
    overwrite=True,
    directory="cnn_search",
    max_trials=50,
    project_name="standard_image"
)

tuner.search_space_summary()
tuner.search(train_dataloader, epochs=30, validation_data=validation_dataloader,
             callbacks=[
                 # keras.callbacks.TensorBoard("/tmp/tb_logs"),
                 keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
             ])

In [3]:
tuner.get_best_hyperparameters()

In [8]:
tuner.results_summary()

In [9]:
tuner.get_best_hyperparameters()
# On first 5 best hyperparameters sets

In [ ]:
from keras_tuner import HyperParameters

## Lets try our best hypermodel
hyperparameters = HyperParameters()
hyperparameters.Fixed("lr", value=1e-3)
hyperparameters.Int(name="yey", min_value=32, max_value=64)
hyperparameters.Fixed(name=f"kernel_0", value=3),
hyperparameters.Fixed(name=f"filters_0", value=32)

hyperparameters.Fixed(name=f"kernel_1", value=3),
hyperparameters.Fixed(name=f"filters_1", value=64)
hyperparameters.Fixed(name="hidden_units", value=64)

In [ ]:
type(hyperparameters.get("yey"))
hyperparameters.Int(name="yey", min_value=3, max_value=22, default=11)

In [ ]:
# todo: trova l'errore. Se uso il KFOLD pare funzionare correttamrente l'apprendimento mentre con i dataset diretti no. Che sia un problema di shuffle? Che sia che i channel non sono invertiti?
hypermodel = SimpleCNNContainer((3, 224, 224))

train, test = dataset_loader((224, 224), is_grayscale=False)

dataset_split_controller = KFoldController(5)
dataset_split_controller.load_data(train)

local_train, validation = dataset_split_controller.get_data_for_fold(0)

train_dataloader = DataLoader(dataset=local_train, batch_size=32)
validation_dataloader = DataLoader(dataset=validation, batch_size=32)

model = hypermodel.build(hyperparameters)
model.summary()

In [ ]:
train_dataloader = DataLoader(dataset=train, batch_size=32, shuffle=True)
validation_dataloader = DataLoader(dataset=test, batch_size=32,shuffle=True)

In [ ]:
from matplotlib import pyplot as plt
figure = plt.figure(figsize=(8, 8))

image, label = train[0]
figure.add_subplot(1,1,image)
plt.title(label)
plt.axis("off")
plt.imshow(image.squeeze(), cmap="gray")
plt.show()

In [ ]:
i_history = model.fit(train_dataloader, epochs=100, validation_data=validation_dataloader,
                      callbacks=[
                          # To avoid keep going
                          keras.callbacks.EarlyStopping(monitor='val_loss', patience=10,
                                                        restore_best_weights=True),

                          # To reduce learning rate (like Hot Stocastic bla bla)
                          #keras.callbacks.LearningRateScheduler(
                          #    learning_rate_reschedule.learning_rate_reschedule)
                      ])